# Terea 1: Reconocedor basado en whisper

## 1.1
Aprovechando lo visto en los cuadernos del taller se propone crear un tokenizador adaptado al vocabulario de la tarea. <br>
La idea es que sea orientado a palabra ya que hay pocas y simplifica el reconocimiento. <br> 
Se puede reaprovechar el tokenizador
DigitSumTokenizer que aparece en el cuaderno: parte2/run11. Implemente el
tokenizador para la tarea: <br>
- fechas2/fechas2_train.es.csv <br>
- fechas2/fechas2_test.es.csv <br>